In [2]:
# version 2018 is quite easy
# download & extract
url = 'https://anaconda.org/rdkit/rdkit/2018.09.1.0/download/linux-64/rdkit-2018.09.1.0-py36h71b666b_1.tar.bz2'
!curl -L $url | tar xj lib
# move to python packages directory
!mv lib/python3.6/site-packages/rdkit /usr/local/lib/python3.6/dist-packages/
x86 = '/usr/lib/x86_64-linux-gnu'
!mv lib/*.so.* $x86/
# rdkit need libboost_python3.so.1.65.1
!ln -s $x86/libboost_python3-py36.so.1.65.1 $x86/libboost_python3.so.1.65.1

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3689    0  3689    0     0  16109      0 --:--:-- --:--:-- --:--:-- 16109
100 20.2M  100 20.2M    0     0  5508k      0  0:00:03  0:00:03 --:--:-- 6781k


In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
ls

drive/  lib/  sample_data/


In [5]:
%cd drive/MyDrive/

/content/drive/MyDrive


In [6]:
cd drug_discovery/

/content/drive/.shortcut-targets-by-id/19b_O5F-YXehQUaR7OjSc5qFpWmOOs-7k/drug_discovery


In [7]:
ls

AA.csv  drug_discovery1.ipynb


In [8]:
import pandas as pd
data = pd.read_csv('/content/drive/My Drive/drug_discovery/AA.csv')
data

,zinc_id,smiles
0,ZINC000000008151,C[C@H]1[C@@H](O)[C@H](CO)O[C@@H](O)[C@@H]1N
1,ZINC000000008153,CC[C@@H]1[C@@H](N)[C@@H](O)O[C@@H](CO)[C@@H]1O
2,ZINC000000008155,CC1(C)[C@@H](N)[C@@H](O)O[C@@H](CO)[C@@H]1O
3,ZINC000000018276,CS[C@@H]1CN[C@@H](CO)[C@H](O)[C@H]1O
4,ZINC000000018279,CS[C@@H]1[C@@H](O)CN[C@@H](CO)[C@@H]1O
...,...,...
427851,ZINC000242463989,O[Cl+3](O)(O)O
427852,ZINC000247713634,O1[SiH2][SiH2]O[SiH2][SiH2]1
427853,ZINC000252581626,O[Si](O)(O)F
427854,ZINC000685945533,Cn1nnnc1S(=O)(=O)F


In [9]:
data['smiles'].apply(len).max()

62

In [10]:
data['smiles'].apply(len).min()

4

In [11]:
from rdkit import Chem

In [38]:
SMILES_CHARS = [' ',
                  '#', '%', '(', ')', '+', '-', '.', '/',
                  '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                  '=', '@',
                  'A', 'B', 'C', 'F', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P',
                  'R', 'S', 'T', 'V', 'X', 'Z',
                  '[', '\\', ']',
                  'a', 'b', 'c', 'e', 'g', 'i', 'l', 'n', 'o', 'p', 'r', 's',
                  't', 'u']
smi2index = dict( (c,i) for i,c in enumerate( SMILES_CHARS ) )
index2smi = dict( (i,c) for i,c in enumerate( SMILES_CHARS ) )
def smiles_encoder( smiles, maxlen=62 ):
    smiles = Chem.MolToSmiles(Chem.MolFromSmiles( smiles ))
    X = np.zeros( ( maxlen, len( SMILES_CHARS ) ) )
    for i, c in enumerate( smiles ):
        X[i, smi2index[c]] = 1
    return X
def smiles_decoder( X ):
    smi = ''
    X = X.argmax( axis=-1 )
    for i in X:
        smi += index2smi[i]
    return smi

In [13]:
import numpy as np
x = []
for i in data['smiles'][:1000]:
    #print(smiles_encoder(i))
    x.append(smiles_encoder(i))
arr = np.array(x)

In [14]:
arr.shape

(1000, 62, 56)

In [15]:
arr[0].shape

(62, 56)

In [16]:
arr = arr.reshape(-1,62,56,1)

In [17]:
arr.shape

(1000, 62, 56, 1)

In [18]:
# #Hyperparameters

# #Number of epochs for training
# NUM_EPOCHS = 35

# #Hidden channel dimension
# hidden_channels = 32

# #Latent space dimension
# latent_dim = 196

# #Convolutional layer kernel size
# kernel_size = 3

# #Stride is the number of pixels shifts over the input matrix  
# stride = 1

# #Padding size to add around the matrix.
# padding = 1

In [19]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches 
%matplotlib inline
from scipy.stats import norm 
import keras
from keras import layers
from keras.models import Model 
from keras import metrics
from keras import backend as K 
K.clear_session()
np.random.seed(237)
import scipy.stats as st
import tensorflow.compat.v1.keras.backend as K 
import tensorflow as tf 
tf.compat.v1.disable_eager_execution()
from tensorflow.keras.optimizers import Adam 
import warnings 
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
import numpy as np
from scipy.stats import multivariate_normal

In [21]:
def sampling(args):
  z_mu, z_log_sigma = args
  epsilon = K.random_normal(shape=(K.shape(z_mu)[0], latent_dim),
  mean=0., stddev=1.)
  return z_mu + K.exp(z_log_sigma) * epsilon
z = layers.Lambda(sampling)([z_mu, z_log_sigma])

In [25]:
img_shape = (62, 56,1)
batch_size = 16
latent_dim = 2
input_img = tf.keras.Input(shape=img_shape)

x = layers.Conv1D(32, 7,
                  strides = 2)(input_img)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

x = layers.Conv1D(64, 7,
                  
                  strides = 2)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

x = layers.Conv1D(128, 7,
                  
                  strides = 2)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

x = layers.Dense(512)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

shape_before_flattening = K.int_shape(x)
x = layers.Flatten()(x)
z_mu = layers.Dense(latent_dim)(x)
z_log_sigma = layers.Dense(latent_dim)(x)
z = layers.Lambda(sampling)([z_mu, z_log_sigma])

encoder = tf.keras.Model(input_img,[z_mu,z_log_sigma])
encoder.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 62, 56, 1)]  0                                            
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 62, 25, 32)   256         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_8 (BatchNor (None, 62, 25, 32)   128         conv1d_6[0][0]                   
__________________________________________________________________________________________________
activation_8 (Activation)       (None, 62, 25, 32)   0           batch_normalization_8[0][0]      
____________________________________________________________________________________________

In [26]:
def sampling(args):
  z_mu, z_log_sigma = args
  epsilon = K.random_normal(shape=(K.shape(z_mu)[0], latent_dim),
  mean=0., stddev=1.)
  return z_mu + K.exp(z_log_sigma) * epsilon
z = layers.Lambda(sampling)([z_mu, z_log_sigma])

In [27]:
decoder_input = layers.Input(K.int_shape(z)[1:])
x = layers.Dense(np.prod(shape_before_flattening[1:]),activation='relu')(decoder_input)
#Decoder
x = layers.Dense(512)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

x = layers.RepeatVector(62)(x)

x = tf.compat.v1.keras.layers.CuDNNGRU(512, return_sequences=True)(x)
x = layers.BatchNormalization()(x)

x = tf.compat.v1.keras.layers.CuDNNGRU(512, return_sequences=True)(x)
x = layers.BatchNormalization()(x)

x = tf.compat.v1.keras.layers.CuDNNGRU(512, return_sequences=True)(x)

x = layers.TimeDistributed(layers.Dense(56))(x)
x = layers.Activation('softmax')(x)

In [28]:
decoder = Model(decoder_input, x)
z_decoded = decoder(z)

In [29]:
decoder.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense_9 (Dense)              (None, 63488)             190464    
_________________________________________________________________
dense_10 (Dense)             (None, 512)               32506368  
_________________________________________________________________
batch_normalization_12 (Batc (None, 512)               2048      
_________________________________________________________________
activation_12 (Activation)   (None, 512)               0         
_________________________________________________________________
repeat_vector (RepeatVector) (None, 62, 512)           0         
_________________________________________________________________
cu_dnngru (CuDNNGRU)         (None, 62, 512)           1575

In [30]:
# decoder_input = layers.Input(K.int_shape(z)[1:])
# x = layers.Dense(np.prod(shape_before_flattening[1:]),
#                  activation='relu')(decoder_input)
# x = layers.Reshape(shape_before_flattening[1:])(x)
# x = layers.Conv2DTranspose(32, 3,
#                            padding='same',
#                            activation='relu',
#                            strides=(2, 2))(x)
# x = layers.Conv2D(1, 3,
#                   padding='same',
#                   activation='sigmoid')(x)
# decoder = Model(decoder_input, x)
# z_decoded = decoder(z)

In [31]:
class CustomVariationalLayer(keras.layers.Layer):
    def vae_loss(self, x, z_decoded):
        x = K.flatten(x)
        z_decoded = K.flatten(z_decoded)
        xent_loss = keras.metrics.binary_crossentropy(x, z_decoded) #Recin loss
        kl_loss = -5e-4 * K.mean(1 + z_log_sigma - K.square(z_mu) - K.exp(z_log_sigma), axis=-1) #KL loss
        return K.mean(xent_loss + kl_loss)

    def call(self, inputs):
        x = inputs[0]
        z_decoded = inputs[1]
        loss = self.vae_loss(x, z_decoded)
        self.add_loss(loss, inputs=inputs)
        return x

y = CustomVariationalLayer()([input_img, z_decoded])

In [32]:
# VAE model statement
vae = Model(input_img, y)
vae.compile(optimizer='rmsprop', loss=None)
vae.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 62, 56, 1)]  0                                            
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 62, 25, 32)   256         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_8 (BatchNor (None, 62, 25, 32)   128         conv1d_6[0][0]                   
__________________________________________________________________________________________________
activation_8 (Activation)       (None, 62, 25, 32)   0           batch_normalization_8[0][0]      
____________________________________________________________________________________________

In [33]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)
vae.fit(x=arr, y=None,
        shuffle=True,
        epochs=120,
        batch_size=batch_size,callbacks=[callback]
        )

Train on 1000 samples
Epoch 1/120
1000/1000 [==============================] - 24s 24ms/sample - loss: 12976.5682
Epoch 2/120
1000/1000 [==============================] - 2s 2ms/sample - loss: 4.8044
Epoch 3/120
1000/1000 [==============================] - 2s 2ms/sample - loss: 1.1516
Epoch 4/120
1000/1000 [==============================] - 2s 2ms/sample - loss: 0.1252
Epoch 5/120
1000/1000 [==============================] - 2s 2ms/sample - loss: 0.8886
Epoch 6/120
1000/1000 [==============================] - 2s 2ms/sample - loss: 8.9856
Epoch 7/120
1000/1000 [==============================] - 2s 2ms/sample - loss: 0.3029
Epoch 8/120
1000/1000 [==============================] - 2s 2ms/sample - loss: 76769.2847
Epoch 9/120
1000/1000 [==============================] - 2s 2ms/sample - loss: 15.2412
Epoch 10/120
1000/1000 [==============================] - 2s 2ms/sample - loss: 17.1902
Epoch 11/120
1000/1000 [==============================] - 2s 2ms/sample - loss: 1423251.7075
Epoch 12/120

In [34]:

sample_vector = np.array([[0,5]])
result = decoder.predict(sample_vector)

In [35]:
result.shape

(1, 62, 56)

In [ ]:
result = result[0].reshape(62,56)

In [36]:
result.shape

(1, 62, 56)

In [40]:
smiles_decoder(result[0])

'CCC@O)CCCCCCCOOOOOO@@@@C]=CCCCCC@@@@@@@@@@11111111111111111111'

In [ ]:
# import rdkit
# rdkit.Chem.MolDrawing(smiles_decoder(result))

In [ ]:
from rdkit.Chem import Draw
m = Chem.MolFromSmiles(smiles_decoder(result))
fig = Draw.MolToMPL(m, size=(200,200))

ValueError: ignored